In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

In [2]:
# Print current directory and contents
print("Current working directory:", os.getcwd())
print("Contents of current directory:", os.listdir())

Current working directory: c:\Users\Dell\Desktop\Task2Code\INM702
Contents of current directory: ['.git', 'coursework_task_1.ipynb', 'engine_data.csv', 'main.ipynb', 'README.md', 'Task2codefile.ipynb', 'updated_pollution_dataset.csv']


In [3]:
base_dir = r'c:\Users\Dell\Desktop\Task2Code'

In [4]:
# Create dataset paths 
train_path = os.path.join('Train', 'Train')
valid_path = os.path.join('Validation', 'Validation')
test_path = os.path.join('Test', 'Test')

In [8]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()  
])

In [9]:
# Create dataset paths
train_path = os.path.join(base_dir, 'Train', 'Train')
valid_path = os.path.join(base_dir, 'Validation', 'Validation')
test_path = os.path.join(base_dir, 'Test', 'Test')

In [10]:

# Create datasets
train_dataset = datasets.ImageFolder(train_path, transform=transform)
valid_dataset = datasets.ImageFolder(valid_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)


In [11]:
# Print dataset information
print(f'Number of training images: {len(train_dataset)}')
print(f'Number of validation images: {len(valid_dataset)}')
print(f'Number of test images: {len(test_dataset)}')
print(f'Classes: {train_dataset.classes}')


Number of training images: 1322
Number of validation images: 60
Number of test images: 150
Classes: ['Healthy', 'Powdery', 'Rust']


In [12]:
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

BaseLine Model

In [13]:
class BaselineCNN(nn.Module):
    def __init__(self):
        super(BaselineCNN, self).__init__()
        # Just one conv layer
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Two fully connected layers
        self.fc1 = nn.Linear(16 * 16 * 16, 64)
        self.fc2 = nn.Linear(64, 3)  # 3 classes
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
# Initialize model and training components
model = BaselineCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

Train the baseline model

In [17]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Simple training components
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

print("Starting training...")
num_epochs = 5 

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_acc = 100. * correct / total
    
    # Validation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    val_acc = 100. * correct / total
    
    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Loss: {running_loss/len(train_loader):.3f}')
    print(f'Train Accuracy: {train_acc:.2f}%')
    print(f'Validation Accuracy: {val_acc:.2f}%\n')

# Simple evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f'Test Accuracy: {100.*correct/total:.2f}%')

Starting training...
Epoch 1/5:
Loss: 0.830
Train Accuracy: 65.20%
Validation Accuracy: 46.67%

Epoch 2/5:
Loss: 0.773
Train Accuracy: 68.15%
Validation Accuracy: 46.67%

Epoch 3/5:
Loss: 0.762
Train Accuracy: 68.91%
Validation Accuracy: 35.00%

Epoch 4/5:
Loss: 0.706
Train Accuracy: 71.86%
Validation Accuracy: 46.67%

Epoch 5/5:
Loss: 0.713
Train Accuracy: 69.29%
Validation Accuracy: 58.33%

Test Accuracy: 56.67%


In [18]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = 100. * correct / total 
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 56.67%


In [19]:
baseline_results = {
    'test_accuracy': test_accuracy,  
    'model_state': model.state_dict().copy()
}


print(f"Baseline Test Accuracy: {float(baseline_results['test_accuracy']):.2f}%")



baseline_transform = transform  
baseline_model = model  

Baseline Test Accuracy: 56.67%


## Implement Improvement 1

- Larger input size (224x224) for better feature detection
- Data augmentation with random flips for better generalization
- ImageNet normalization for training stability

In [20]:
import torch
from torchvision import transforms

In [21]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),          # Increased from 32x32
    transforms.RandomHorizontalFlip(),      # New: data augmentation
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])  # Added normalization
])


In [22]:
# Updated datasets with new transform
train_dataset = datasets.ImageFolder(train_path, transform=transform)
valid_dataset = datasets.ImageFolder(valid_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

In [23]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

IMPROVEMENT 2: Enhanced CNN Architecture
Changes from your advanced model:
- Three convolutional layers instead of one
- Added padding to maintain spatial dimensions
- Increased number of filters (32, 64, 128)
- Deeper network for better feature learning
"""

## Implement Improvement 2


- Three convolutional layers instead of one
- Added padding to maintain spatial dimensions
- Increased number of filters (32, 64, 128)


In [24]:
class PlantDiseaseCNN(nn.Module):
    def __init__(self, num_classes):
        super(PlantDiseaseCNN, self).__init__()
        
         
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        
        x = x.view(-1, 128 * 28 * 28)
        
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Initialize model with improvements
num_classes = len(train_dataset.classes)
model = PlantDiseaseCNN(num_classes)

## Implement improvement 3


- Using Adam optimizer instead of SGD
- Added dropout for regularization
- More epochs (10 instead of 5)


In [25]:

# Training components
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Changed to Adam

# Training loop
print("Starting training...")
num_epochs = 10  # epochs increased
best_val_acc = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    # Calculate metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total

    # Validation phase
    model.eval()
    valid_correct = 0
    valid_total = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            valid_correct += (predicted == labels).sum().item()
            valid_total += labels.size(0)

    valid_acc = valid_correct / valid_total

    # Save best model
    if valid_acc > best_val_acc:
        best_val_acc = valid_acc
        torch.save(model.state_dict(), 'best_plant_disease_model.pth')

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Loss: {epoch_loss:.4f}")
    print(f"Train Accuracy: {epoch_acc*100:.2f}%")
    print(f"Valid Accuracy: {valid_acc*100:.2f}%\n")

Starting training...
Epoch 1/10
Loss: 1.1100
Train Accuracy: 59.30%
Valid Accuracy: 61.67%

Epoch 2/10
Loss: 0.4233
Train Accuracy: 83.96%
Valid Accuracy: 81.67%

Epoch 3/10
Loss: 0.2438
Train Accuracy: 91.45%
Valid Accuracy: 81.67%

Epoch 4/10
Loss: 0.1596
Train Accuracy: 95.39%
Valid Accuracy: 85.00%

Epoch 5/10
Loss: 0.1679
Train Accuracy: 94.63%
Valid Accuracy: 90.00%

Epoch 6/10
Loss: 0.1768
Train Accuracy: 93.87%
Valid Accuracy: 93.33%

Epoch 7/10
Loss: 0.0990
Train Accuracy: 96.67%
Valid Accuracy: 91.67%

Epoch 8/10
Loss: 0.0872
Train Accuracy: 97.43%
Valid Accuracy: 95.00%

Epoch 9/10
Loss: 0.1066
Train Accuracy: 97.28%
Valid Accuracy: 90.00%

Epoch 10/10
Loss: 0.1085
Train Accuracy: 96.14%
Valid Accuracy: 93.33%



In [26]:
# Load the best model
model.load_state_dict(torch.load('best_plant_disease_model.pth'))

# Evaluation function
def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy*100:.2f}%")

evaluate_model(model, test_loader)

C:\Users\Dell\AppData\Local\Temp\ipykernel_24040\2132295733.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_plant_disease_model.pt

Test Accuracy: 92.00%
